# Make your requests faster

When you start scraping web-pages or requesting APIs, you will be facing a problem when doing a lot of requests: this is really slow!

It's because python is slow! You could say, well it should not be. Let's see how we can speed things up!

## Measure performances

In this notebook we will need to track how much time some code is taking to execute.
To make things easier, we will create a simple decorator that will print the number of micro-seconds a function takes to execute.

A good opportonity to practice decorators in a practical example!

*Note that you need python3.3 or higher.*

In [1]:
import time

def print_timing(func):
    '''Create a timing decorator function use @print_timing just above the function you want to time.'''

    def wrapper(*arg):
        start = time.perf_counter()
        
        # Run the function decorated
        result = func(*arg)

        end = time.perf_counter()
        execution_time = round((end - start), 2)
        print(f'{func.__name__} took {execution_time} sec')
        return result

    return wrapper


@print_timing
def example():
    time.sleep(2)


example()

example took 2.0 sec


## The API

For this example, we will use the [quotable.io](https://api.quotable.io) API. It's an online API you can use to generate random quote.

But feel free to replace `api_url` value with any API you'd like.

In [2]:
api_url = "https://api.quotable.io/random"

## The "classic" way

If you start playing with requests, your should probably have something like this:

In [3]:
import requests


def basic_request(url: str):
    response = requests.get(url)
    response_json = response.json()
    print(response_json["content"])


@print_timing
def basic_loop_request(url: str):
    # Query 50 times the API
    for _ in range(50):
        basic_request(url)


basic_loop_request(api_url)

It is not fair to ask of others what you are unwilling to do yourself.
Happiness is found in doing, not merely possessing.
All men have a sweetness in their life. That is what helps them go on. It is towards that they turn when they feel too worn out.
Our greatest weakness lies in giving up. The most certain way to succeed is always to try just one more time.
As a technology, the book is like a hammer. That is to say, it is perfect: a tool ideally suited to its task. Hammers can be tweaked and varied but will never go obsolete. Even when builders pound nails by the thousand with pneumatic nail guns, every household needs a hammer.
We should not judge people by their peak of excellence; but by the distance they have traveled from the point where they started.
If you live to be a hundred, I want to live to be a hundred minus one day, so I never have to live without you.
Before we acquire great power, we must acquire wisdom to use it well.
Sometimes the most important thing in a whole day

### Results

On my machine it took **17.06 sec for 50 requests**. 

Pretty slow right? But why is that?

Each time you make a request, your computer needs to create a new "session", format your request, send it and wait to receive the response before doing it again with the next request.

## The "session" way

To speed this, we can use a **"session"** that will be share by all the requests.

You can picture it as a postman that knows you already, so he knows which bell to ring, where is the mailbox,... Instead of having to search for those each time.

In [4]:
from requests import Session


def session_request(url: str, session: Session):
    # Instead of using request.get, we use our session
    response = session.get(url)
    response_json = response.json()
    print(response_json["content"])


@print_timing
def session_loop_request(url: str):
    # Create shared session for all of your requests
    with Session() as session:
        # Query 50 times the API
        for _ in range(50):
            session_request(url, session)


session_loop_request(api_url)

Life is what you make of it. Always has been, always will be.
Injuries may be forgiven, but not forgotten.
Constant kindness can accomplish much. As the sun makes ice melt, kindness causes misunderstanding, mistrust, and hostility to evaporate.
The minute you settle for less than you deserve, you get even less than you settled for.
Let us always meet each other with smile, for the smile is the beginning of love.
I never considered a difference of opinion in politics, in religion, in philosophy, as cause for withdrawing from a friend.
When an opponent comes forward, move in and greet him; if he wants to pull back, send him on his way.
My religion consists of a humble admiration of the illimitable superior spirit who reveals himself in the slight details we are able to perceive with our frail and feeble mind.
I think that we have a great opportunity to impart our wisdom and our knowledge and our experience to this younger generation. It may be different times, but experience transcends t

### Results

It took me **5.99 sec for 50 requests**. That's better!

And as you can see, I didn't change that much in the code.

## The "Async" way

If you need even more performances, you will need to use [AsyncIo](https://docs.python.org/3/library/asyncio.html).

This is a library to allow you to run asynchronous code.

Why is that more efficiant? Well, when you send a request you need to wait for the response. And during the waiting time, our computer does nothing.
If you count all the time the computer is just "waiting" on 50 or more requests, you will be surprised to see that most of the computing time is just waiting for the server to respond.

[AsyncIo](https://docs.python.org/3/library/asyncio.html) allow you to bypass that.

But as always, it has a cost: complexity.

Making your code async will complixify the code a lot and make the debugging not a pleasant experience. Also, you will go so fast that you could be banned by the server.

My advice? Use it only if you need it.

I will show you a simple example but you want to understand it better, I really advice you **[this video](https://www.youtube.com/watch?v=qAh5dDODJ5k)**!

### Requirements
In order to simplify a bit the code, I will use [httpx](https://www.python-httpx.org/) a python library that is working the same way as the `requests` module but with few helpers for async.

In [5]:
!pip install httpx

### Warning!
This code won't work in jupyter notebook, there are subtilities for async in jupyter notebook. See [this thread](https://stackoverflow.com/questions/47518874/how-do-i-run-python-asyncio-code-in-a-jupyter-notebook) for more informations.

To make it simpler, I will put this code in a .py file and run it in command line:

```python
from httpx import AsyncClient
import asyncio
import time


api_url = "https://api.quotable.io/random"


async def session_request_async(url: str, session: AsyncClient):
    # Instead of using request.get, we use our session
    response = await session.get(url)
    response_json = response.json()
    print(response_json["content"])
    return response_json


async def session_loop_request_async(url: str):
    # Create shared session for all of your requests
    async with AsyncClient() as session:
        # Create a list of empty tasks
        tasks = []
        # Query 50 times the API
        for _ in range(50):
            # Add a request to tasks
            tasks.append(
                asyncio.create_task(
                    session_request_async(url, session)        
                )
            )
        # Now that all the tasks are registred, run them
        responses = await asyncio.gather(*tasks)
            
            


start = time.perf_counter()

# We need to use asyncio.run to run the async function
asyncio.run(session_loop_request_async(api_url))

end = time.perf_counter()
execution_time = round((end - start), 2)
print(f'session_loop_request_async took {execution_time} sec')
```

In [6]:
!python3 ./assets/async_requests.py

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


### Results
It only took me **0.8 sec for 50 requests**! That's impressive.

But as you can see, it is harder to write, structure and debug. So make sure you **really** need it if you consider using this method.

## Summary

If we gather all our results:

| Method                     | Execution time for 50 requests |
|----------------------------|--------------------------------|
| `requests.get` loop        | 17.06 sec                  |
| `requests` with `Session`  | 5.99 sec                   |
| `httpx` with `AsyncClient` | 0.8 sec                   |